# Library

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm.notebook import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from torchvision import transforms
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from glob import glob
from torch import nn
from PIL import Image
from pathlib import Path
from transformers import AutoModel, AutoProcessor
import cv2
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm.notebook import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

BATCHSIZE=128
SAVE_OPT_CKP = True
SAVE_MODEL_CKP = True
UNFREEZE_START = 18 # set it to lower number when significantly more samples are included.

torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn

run_name = f'open-clip224-l14'
import gc

# Config

In [ ]:
# class CFG:
#     model_path = '/kaggle/input/1742swin/swinv2_large_window12to16_192to256_22kft1k-epoch-2.pth'#
#     model_name = 'swinv2_large_window12to16_192to256_22kft1k'
#     input_size = 256
#     batch_size = 32

# Dataset

In [ ]:
# class DiffusionTestDataset(Dataset):
#     def __init__(self, images, transform):
#         self.images = images
#         self.transform = transform
    
#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         image = Image.open(self.images[idx])
#         image = self.transform(image)
#         return image

# Inference

In [ ]:
# def predict(
#     images,
#     model_path,
#     model_name,
#     input_size,
#     batch_size
# ):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     transform = transforms.Compose([
#         transforms.Resize(input_size),
#         transforms.RandomHorizontalFlip(p=0.5),
# #         transforms.RandomRotation(degrees=10),

#         #transforms.RandomVerticalFlip(p=0.5),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#     ])
#     dataset = DiffusionTestDataset(images, transform)
#     dataloader = DataLoader(
#         dataset=dataset,
#         shuffle=False,
#         batch_size=batch_size,
#         pin_memory=True,
#         num_workers=2,
#         drop_last=False
#     )

#     model = timm.create_model(
#         model_name,
#         pretrained=False,
#         num_classes=384
#     )
# #     state_dict = torch.load(model_path)
#     #model.load_state_dict(state_dict，map_=device)
#     weights = torch.load(model_path)
#     model.load_state_dict(weights)
#     model.to(device)
#     model.eval()
    
#     tta_preds = None
#     for _ in range(3):     #预测两次取平均值
#         preds = []
#         for X in tqdm(dataloader, leave=False):
#             X = X.to(device)

#             with torch.no_grad():
#                 X_out = model(X)
#                 preds.append(X_out.cpu().numpy())
                
#         if tta_preds is None:
#             tta_preds = np.vstack(preds).flatten()    #按行组合
#         else:
#             tta_preds += np.vstack(preds).flatten()
    
#     return tta_preds / 3

In [ ]:
# images = list(Path('/kaggle/input/stable-diffusion-image-to-prompts/images').glob('*.png'))
# imgIds = [i.stem for i in images]
# EMBEDDING_LENGTH = 384
# imgId_eId = [
#     '_'.join(map(str, i)) for i in zip(
#         np.repeat(imgIds, EMBEDDING_LENGTH),
#         np.tile(range(EMBEDDING_LENGTH), len(imgIds)))]

# prompt_embeddings = predict(images, CFG.model_path, CFG.model_name, CFG.input_size, CFG.batch_size)
# submission = pd.DataFrame(
#     index=imgId_eId,
#     data=prompt_embeddings,
#     columns=['val']
# ).rename_axis('imgId_eId')
# submission.to_csv('submission.csv')

In [ ]:
wheels_path = "/kaggle/input/open-clip-wheels/open_clip_wheels"
open_clip_whl_path = f"{wheels_path}/open_clip_torch-2.14.0-py3-none-any.whl"
!pip install --no-index --find-links $wheels_path $open_clip_whl_path -q
import open_clip

In [ ]:
sample_7 = "/kaggle/input/stable-diffusion-image-to-prompts/images"
# sd2 = '/kaggle/input/gustavosta-stable-diffusion-prompts-sd2-v2/eval_images'
path = sample_7

In [ ]:
class DiffusionTestDataset(Dataset):
    def __init__(self, images, transform, input_size):
        self.images = images
        self.transform = transform
        self.size = input_size
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        image.resize((self.size, self.size))
        image = self.transform(image)
        return image

In [ ]:
class Net(nn.Module):
    def __init__(self, model):
        super(Net, self).__init__()
        clip = model
        self.vision = clip.visual
        self.fc = nn.Linear(768, 384)

    def forward(self, x):
        out = self.vision(x)
        return self.fc(out)

def predict(
    images,
    model_path,
    model_name,
    input_size,
    batch_size,
    open_clip_model=False,
):
    
    if not open_clip_model:
        model = timm.create_model(
            model_name,
            pretrained=False,
            num_classes=384
        )
        transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    else:
        model, _, preprocess = open_clip.create_model_and_transforms(model_name)
        model = Net(model)
        transform = preprocess
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    dataset = DiffusionTestDataset(images, transform, input_size)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    
    #MLP = 
    state_dict = torch.load(model_path, map_location='cuda:0')
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    
    preds = []
    for X in tqdm(dataloader, leave=False):
        X = X.to(device)

        with torch.no_grad():
            X_out = model(X)
            preds.append(X_out.cpu().numpy())
    
    return np.vstack(preds).flatten()

In [ ]:
class CFG:
    model_path = '/kaggle/input/one111/60w_convnextl_one-11.pt'
    model_name = 'convnext_large_d'
    model_path1 = '/kaggle/input/20alll/60w_again_hard1_sgd-19.pt'
    model_name1 = 'ViT-L-14'
    model_path3 = '/kaggle/input/1742swin/swinv2_large_window12to16_192to256_22kft1k-epoch-2.pth'#
    model_name3 = 'swinv2_large_window12to16_192to256_22kft1k'
    input_size = 224
    batch_size = 64
    input_size1 = 256
    batch_size1 = 32
images = list(Path(path).glob('*.jpg'))
images.extend(list(Path(path).glob('*.png')))
images = sorted(images)
prompt_embeddings_clip_vitL = predict(images, CFG.model_path, CFG.model_name, CFG.input_size, CFG.batch_size, open_clip_model=True)
prompt_embeddings_clip_vitL2 = predict(images, model_path=CFG.model_path3, model_name=CFG.model_name3 , input_size=CFG.input_size1, batch_size=CFG.batch_size1)
prompt_embeddings_clip_vitL1 = predict(images, model_path=CFG.model_path1, model_name=CFG.model_name1 , input_size=CFG.input_size, batch_size=CFG.batch_size, open_clip_model=True)

In [ ]:
prompt_embeddings = prompt_embeddings_clip_vitL*0.25+prompt_embeddings_clip_vitL1*0.55+prompt_embeddings_clip_vitL2*0.2

In [ ]:

imgIds = [i.stem for i in images]
EMBEDDING_LENGTH = 384
imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, EMBEDDING_LENGTH),
        np.tile(range(EMBEDDING_LENGTH), len(imgIds)))]

submission = pd.DataFrame(
    index=imgId_eId,
    data=prompt_embeddings,
    columns=['val']
).rename_axis('imgId_eId')
submission.to_csv('submission.csv')

In [ ]:
# import sys
# sys.path.append('../input/sentence-transformers-222/sentence-transformers')
# from sentence_transformers import SentenceTransformer, models
# st_model = SentenceTransformer('/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2')

# comp_path = Path('/kaggle/input/gustavosta-stable-diffusion-prompts-sd2-v2/')
# prompts = pd.read_csv(comp_path / 'eval.csv', index_col='image_path')
# sample_prompt_embeddings = st_model.encode(prompts['Prompt']).flatten()

In [ ]:
# from scipy import spatial
# def cosine_similarity(y_trues, y_preds):
#     return 1 - spatial.distance.cosine(y_trues, y_preds) 

# sample_cos = cosine_similarity(prompt_embeddings, sample_prompt_embeddings)
# print(sample_cos)